# Chrome Dino Game Reinforcement Learning Infrastructure
- Check out the following [github](https://luungoc2005.github.io/blog/2020-06-15-chrome-dino-game-reinforcement-learning/) where I obtained much of the code from (selenium).
- Testing out multiple split screen for getting data inputs (dinosaur frames)

In [6]:
import os
import time
import gym
from gym import spaces

from selenium import webdriver # get webdriver.
from selenium.webdriver.common.keys import Keys # Keyboard actions
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
_chrome_options = webdriver.ChromeOptions()
_chrome_options.add_argument("--mute-audio") # Cause we don't want to hear the 100 milestone and jump actions
_chrome_options.add_argument("--disable-gpu")
_chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

serv_obj = Service("chrome_driver/chromedriver") # File name to chrome driver.

_driver = webdriver.Chrome(
    service= serv_obj,
    options=_chrome_options
)

# Wait for everything to load.
time.sleep(2)

try:
    _driver.get('chrome://dino')
except WebDriverException:
    pass

# Start Game
_driver.find_element(By.TAG_NAME,"body").send_keys(Keys.SPACE) # one space to start game.

# Creating the Environment

Here, we create a gym environment for our agent/models to train in (Capture basic actions)

In [8]:
class CustomEnv(gym.Env):
    """A class to control the custom environment for our agents. """
    
    def __init__(self, action_space_size=3):
        # These can be anything from the spaces module -- dig in later.
        self.action_space = spaces.Discrete(action_space_size)
        self.observation_space = spaces.Box()
        
    def step(self, action):
        """ return observation, reward, done, extra_info """
        pass
    
    def render(self, mode='human'):
        """ return image array """
        pass
    
    def reset(self):
        """ Reset the environment."""
        pass
    
    def close(self):
        pass